# PageRank

In this notebook, we will use both NetworkX and cuGraph to compute the PageRank of each vertex in our test dataset.  The NetworkX and cuGraph processes will be interleaved so that each step can be compared.

Notebook Credits
* Original Authors: Bradley Rees and James Wyles
* Created:   08/13/2019
* Updated:   04/06/2022

RAPIDS Versions: 22.04  

Test Hardware

* GV100 32G, CUDA 11.5


## Introduction
Pagerank is measure of the relative importance, also called centrality, of a vertex based on the relative importance of it's neighbors.  PageRank was developed by Google and is (was) used to rank it's search results. PageRank uses the connectivity information of a graph to rank the importance of each vertex. 

See [Wikipedia](https://en.wikipedia.org/wiki/PageRank) for more details on the algorithm.

To compute the Pagerank scores for a graph in cuGraph we use:<br>

**cugraph.pagerank(G,alpha=0.85, max_iter=100, tol=1.0e-5)**
* __G__: cugraph.Graph object
* __alpha__: float, The damping factor represents the probability to follow an outgoing edge. default is 0.85
* __max_iter__: int, The maximum number of iterations before an answer is returned. This can be used to limit the execution time and do an early exit before the solver reaches the convergence tolerance. If this value is lower or equal to 0 cuGraph will use the default value, which is 100
* __tol__: float, Set the tolerance the approximation, this parameter should be a small magnitude value. The lower the tolerance the better the approximation. If this value is 0.0f, cuGraph will use the default value which is 0.00001. Setting too small a tolerance can lead to non-convergence due to numerical roundoff. Usually values between 0.01 and 0.00001 are acceptable.

Returns:
* __df__: a cudf.DataFrame object with two columns:
    * df['vertex']: The vertex identifier for the vertex
    * df['pagerank']: The pagerank score for the vertex


### Some notes about vertex IDs...
* The current version of cuGraph requires that vertex IDs be representable as 32-bit integers, meaning graphs currently can contain at most 2^32 unique vertex IDs. However, this limitation is being actively addressed and a version of cuGraph that accommodates more than 2^32 vertices will be available in the near future.
* cuGraph will automatically renumber graphs to an internal format consisting of a contiguous series of integers starting from 0, and convert back to the original IDs when returning data to the caller. If the vertex IDs of the data are already a contiguous series of integers starting from 0, the auto-renumbering step can be skipped for faster graph creation times.
  * To skip auto-renumbering, set the `renumber` boolean arg to `False` when calling the appropriate graph creation API (eg. `G.from_cudf_edgelist(gdf_r, source='src', destination='dst', renumber=False)`).
  * For more advanced renumbering support, see the examples in `structure/renumber.ipynb` and `structure/renumber-2.ipynb`


### Test Data
We will be using the Zachary Karate club dataset 
*W. W. Zachary, An information flow model for conflict and fission in small groups, Journal of
Anthropological Research 33, 452-473 (1977).*


![Karate Club](../img/zachary_black_lines.png)


### Prep

In [ ]:
# The notebook compares cuGraph to NetworkX,  
# therefore there some additional non-RAPIDS python libraries need to be installed. 
# Please run this cell if you need the additional libraries
%pip install networkx
%pip install scipy

In [ ]:
# Import needed libraries
import cugraph
import cudf
from collections import OrderedDict

In [ ]:
# NetworkX libraries
import networkx as nx
from scipy.io import mmread

### Some Prep

In [ ]:
# define the parameters 
max_iter = 100  # The maximum number of iterations
tol = 0.00001   # tolerance
alpha = 0.85    # alpha

In [ ]:
# Define the path to the test data  
datafile='../data/karate-data.csv'

---
# NetworkX

In [ ]:
# Read the data, this also creates a NetworkX Graph 
file = open(datafile, 'rb')
Gnx = nx.read_edgelist(file)

In [ ]:
pr_nx = nx.pagerank(Gnx, alpha=alpha, max_iter=max_iter, tol=tol)

In [ ]:
pr_nx

Running NetworkX is that easy.  
Let's see how that compares to cuGraph

----

# cuGraph

### Read in the data - GPU
cuGraph graphs can be created from cuDF, dask_cuDF and Pandas dataframes

The data file contains an edge list, which represents the connection of a vertex to another.  The `source` to `destination` pairs is in what is known as Coordinate Format (COO).  In this test case, the data is just two columns.  However a third, `weight`, column is also possible

In [ ]:
# Read the data  
gdf = cudf.read_csv(datafile, names=["src", "dst"], delimiter='\t', dtype=["int32", "int32"] )

### Create a Graph 

In [ ]:
# create a Graph using the source (src) and destination (dst) vertex pairs from the Dataframe 
G = cugraph.from_edgelist(gdf, source='src', destination='dst')

### Call the PageRank algorithm

In [ ]:
# Call cugraph.pagerank to get the pagerank scores
gdf_page = cugraph.pagerank(G)

_It was that easy!_  
Compared to NetworkX, the cuGraph data loading might have been more steps, but using cuDF allows for a wider range of data to be loaded. 


----

Let's now look at the results

In [ ]:
# Find the most important vertex using the scores
# These methods should only be used for small graph
bestScore = gdf_page['pagerank'][0]
bestVert = gdf_page['vertex'][0]

for i in range(len(gdf_page)):
    if gdf_page['pagerank'].iloc[i] > bestScore:
        bestScore = gdf_page['pagerank'].iloc[i]
        bestVert = gdf_page['vertex'].iloc[i]
        
print("Best vertex is " + str(bestVert) + " with score of " + str(bestScore))

The top PageRank vertex and score match what was found by NetworkX

In [ ]:
# A better way to do that would be to find the max and then use the values in a query
pr_max = gdf_page['pagerank'].max()

In [ ]:
def print_pagerank_threshold(_df, t=0) :
    filtered = _df.query('pagerank >= @t')
    
    for i in range(len(filtered)):
        print("Best vertex is " + str(filtered['vertex'].iloc[i]) + 
            " with score of " + str(filtered['pagerank'].iloc[i]))              

In [ ]:
print_pagerank_threshold(gdf_page, pr_max)

----

a PageRank score of _0.10047_ is very low, which can be an indication that there is no more central vertex than any other.  Rather than just looking at the top score, let's look at the top three vertices and see if there are any insights that can be inferred.  

Since this is a very small graph, let's just sort and get the first three records

In [ ]:
sort_pr = gdf_page.sort_values('pagerank', ascending=False)

In [ ]:
sort_pr.head(3)

Going back and looking at the graph with the top three vertices highlighted (illustration below) it is easy to see that the top scoring vertices also appear to be the vertices with the most connections.   
Let's look at sorted list of degrees (since the graph is undirected and symmetrized, the out degree is the same as the in degree)

In [ ]:
d = G.degrees()

In [ ]:
d.sort_values('out_degree', ascending=False).head(3)

<img src="../img/zachary_graph_pagerank.png" width="600">

----
# Personalized PageRank

The issue with PageRank is that it sets the initial weights of all the nodes the same. In other words, it assumes a uniform starting probability for every node. What if we have a priori information about the nodes? We can use personalized PageRank (PPR) to assist.

In [ ]:
# Let's bump up some weights and see how that changes the results
personalization_vec = cudf.DataFrame()
personalization_vec['vertex'] = [17, 26]
personalization_vec['values'] = [0.5, 0.75]

In [ ]:
ppr = cugraph.pagerank(G, alpha=0.85, personalization=personalization_vec, max_iter=100, tol=1.0e-5, nstart=None)

In [ ]:
ppr.sort_values('pagerank', ascending=False).head(3)

In [ ]:
# looking at the initial PageRank values
gdf_page[gdf_page['vertex'].isin([26,17,1])]

___
Copyright (c) 2019-2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___